## 6.5. 汇聚层

通常当处理图像时，希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

机器学习任务通常会跟全局图像的问题有关。所以希望**最后一层的神经元**能够对**整个输入图像**是全局敏感的。

现实中，拍摄照片时，同一时间地点角度拍摄同一个物体都几乎无法保证前后两次拍摄能完美使目标物体保持在完全相同的像素位置

故，引入汇聚层/池化层 pooling的目的是：

1. 降低卷积层对位置的敏感性（不要求前后物体出现在完全一致的像素位置）
2. 降低对空间降采样表示的敏感性（不会因为分辨率/像素信息的减少而使结果差别巨大）

### 6.5.1. 最大汇聚层和平均汇聚层

最大池化层 maximum pooling 和 平均池化层 average pooling

汇聚窗口形状为$p \times q$的汇聚层称为$p \times q$汇聚层，汇聚操作称为$p \times q$汇聚

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2)), pool2d(X, (2, 2), 'avg')

(tensor([[4., 5.],
         [7., 8.]]),
 tensor([[2., 3.],
         [5., 6.]]))

### 6.5.2. 填充和步幅

深度学习框架中汇聚窗口的步幅默认于汇聚窗口的大小相同。

即，汇聚窗口大小为$(3,3)$则步幅同样为$(3,3)$，但也可以手动指定填充和步幅

In [14]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X, X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]]]),
 torch.Size([1, 1, 4, 4]))

In [5]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [6]:
pool2d = nn.MaxPool2d((2,3), stride=(2,3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### 6.5.3. 多个通道

汇聚层在每个输入通道上**单独运算**，汇聚层的输出通道数与输入通道数相同

In [7]:
X = torch.cat((X, X+1), 1)
X, X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 torch.Size([1, 2, 4, 4]))

In [8]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
Y = pool2d(X)
Y, Y.shape

(tensor([[[[ 5.,  7.],
           [13., 15.]],
 
          [[ 6.,  8.],
           [14., 16.]]]]),
 torch.Size([1, 2, 2, 2]))

### 6.5.4. 小结

最大汇聚层，平均汇聚层

汇聚层的主要优点之一是减轻卷积层对位置的过度敏感

可以指定汇聚层的填充和步幅

最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）

汇聚层的输出通道数与输入通道数相同

In [18]:

def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] -h + 1, X.shape[1] -w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).mean()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[1.0, 1.0], [1.0, 1.0]])
corr2d(X, K)

tensor([[2., 3.],
        [5., 6.]])

In [11]:
X.shape,  K.shape

(torch.Size([3, 3]), torch.Size([2, 2]))

In [17]:
pool2d_mean = nn.AvgPool2d(2, padding=0, stride=1)
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d_mean(X.reshape((1, 1, 3, 3)))

tensor([[[[2., 3.],
          [5., 6.]]]])

In [19]:
def maxpool2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] -h + 1, X.shape[1] -w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).max()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[1.0, 1.0], [1.0, 1.0]])
maxpool2d(X, K)

tensor([[4., 5.],
        [7., 8.]])